In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
import scipy as sp
from scipy import signal
from scipy import stats

import warnings
warnings.filterwarnings("ignore")


import scipy.signal as scisig
import matplotlib 
from scipy.integrate import simps
matplotlib.rcParams.update({'figure.max_open_warning': 0})
import scikit_posthocs as spost
#%matplotlib notebook
from sympy import Point, Line, pi
import re

import tkinter as tk
import glob
from tkinter import filedialog as fd
import os

from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations

from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib import cm

import scikit_posthocs as spost

## Parameters

In [ ]:
joints = ["body-coxa","coxa-femur","femur-tibia","tibia-tarsus","claw"]
legs = ["T1","T2","T3"]
angles = ["coxa-femur","femur-tibia","tibia-tarsus"]
cutoff = 2 #s, the time after the stim within which we consider the fly to be reacting to the stim
frame_rate = 120
poststim = 1 #s the amount of time we're counting as the 'uncoordinated phase'
second_start = 3 #s the beginning of where we analyse post-stim as the 'coordinated' phase
second_end = 5 #s the end of where we analyse post-stim as the 'coordinated' phase

#colours
walk = '#4C87C6'
si = '#E21E26'
sl = '#0F8140'
dec = '#984F9F'

## Functions

In [ ]:
def leg_angles(datah,filename,graph='no'): #takes the dataframe of one fly, finds all the angles between the coxa-femur, femur-tibia and the tibia-tarsus, concatenates them into a dataframe
    
    ang1 = []
    ang2 = []
    ang3 = []
    
    mapz1 = []
    mapz2 = []
    mapz3 = []
    
    g = datah.shape[0]
        
    #print(mapz1) 
    
    for i in tqdm(range(g)): #tqdm is the timer
        #print(i)
        
        #coxa-femur
        xupper = np.array(datah["T1-body-coxa"])[i] #(x1)
        xshared = np.array(datah["T1-coxa-femur"])[i] #(xorigin)
        xlower = np.array(datah["T1-femur-tibia"])[i] #(x2)
        yupper = np.array(datah["T1-body-coxa.1"])[i] #(y1)
        yshared = np.array(datah["T1-coxa-femur.1"])[i] #(yorigin)
        ylower = np.array(datah["T1-femur-tibia.1"])[i] #(y2)
        
        if graph == 'yes':
            for i in range(g):
                col1 = [i/g,i/(2*g),1-i/g]
                col2 = [1-i/g,i/g,i/g]
                col3 = [i/(g),1-i/g,i/g]
                mapz1.append(col1)
                mapz2.append(col2)
                mapz3.append(col3)
            
            fig, ax = plt.subplots(figsize = [10,6])
            plt.plot([xshared,xupper],[yshared,yupper])
            plt.plot([xshared,xlower],[yshared,ylower])
            plt.show()
    
        l1 = Line((xshared, yshared), (xupper, yupper))
        l2 = Line((xshared, yshared), (xlower, ylower))
    
        rad1 = l1.angle_between(l2)
        deg1 = math.degrees(rad1)
        ang1.append(deg1)
        
        #print(ang1)
        #print("coxa-femur angle = ",str(math.degrees(rad)))
        
        
        
        #femur-tibia
        xupper = np.array(datah["T1-coxa-femur"])[i] #(x1)
        xshared = np.array(datah["T1-femur-tibia"])[i] #(xorigin)
        xlower = np.array(datah["T1-tibia-tarsus"])[i] #(x2)
        yupper = np.array(datah["T1-coxa-femur.1"])[i] #(y1)
        yshared = np.array(datah["T1-femur-tibia.1"])[i] #(yorigin)
        ylower = np.array(datah["T1-tibia-tarsus.1"])[i] #(y2)
        
        if graph == 'yes':
            for i in range(g):
                col1 = [i/g,i/(2*g),1-i/g]
                col2 = [1-i/g,i/g,i/g]
                col3 = [i/(g),1-i/g,i/g]
                mapz1.append(col1)
                mapz2.append(col2)
                mapz3.append(col3)
            
            #fig, ax = plt.subplots(figsize = [10,6])
            plt.plot([xshared,xupper],[yshared,yupper])
            plt.plot([xshared,xlower],[yshared,ylower])
            plt.show()
    
        l1 = Line((xshared, yshared), (xupper, yupper))
        l2 = Line((xshared, yshared), (xlower, ylower))
    
        rad2 = l1.angle_between(l2)
        deg2 = math.degrees(rad2)
        ang2.append(deg2)
        
        #print(ang1)
        #print("femur-tibia angle = ",str(math.degrees(rad)))
        
        #tibia-tarsus  
        xupper = np.array(datah["T1-femur-tibia"])[i] #(x1)
        xshared = np.array(datah["T1-tibia-tarsus"])[i] #(xorigin)
        xlower = np.array(datah["T1-claw"])[i] #(x2)
        yupper = np.array(datah["T1-femur-tibia.1"])[i] #(y1)
        yshared = np.array(datah["T1-tibia-tarsus.1"])[i] #(yorigin)
        ylower = np.array(datah["T1-claw.1"])[i] #(y2)
        
        if graph == 'yes':
            plt.plot([xshared,xupper],[yshared,yupper])
            plt.plot([xshared,xlower],[yshared,ylower])
            plt.show()
    
        l1 = Line((xshared, yshared), (xupper, yupper))
        l2 = Line((xshared, yshared), (xlower, ylower))
    
        rad3 = l1.angle_between(l2)
        deg3 = math.degrees(rad3)
        ang3.append(deg3)
        
        #print(ang2)
        #print("tibia-tarsus angle = ",str(math.degrees(rad)))
    times = np.array(datah.Timepoint)
    
    f = pd.DataFrame(ang1,columns = ["value"])
    f["angle"] = 1
    f["timepoints"] = times
    f["filename"] = filename  
    if f.timepoints.min() < 0:
        baseline = f.query("timepoints > -0.5 and timepoints < 0").value.mean() #take as a baseline value the 0.5s before the stim
        f["normalised1"] = ang1/baseline
        f["normalised2"] = ang1 - baseline
    
    
    g = pd.DataFrame(ang2,columns = ["value"])
    g["angle"] = 2
    g["timepoints"] = times
    g["filename"] = filename
    if g.timepoints.min() < 0:
        baselineg = g.query("timepoints > -0.5 and timepoints < 0").value.mean() #take as a baseline value the 0.5s before the stim
        g["normalised1"] = ang2/baselineg
        g["normalised2"] = ang2 - baselineg
    
    h = pd.DataFrame(ang3,columns = ["value"])
    h["angle"] = 3
    h["timepoints"] = times
    h["filename"] = filename
    if h.timepoints.min() < 0:
        baselineh = baselineh = h.query("timepoints > -0.5 and timepoints < 0").value.mean() #take as a baseline value the 0.5s before the stim
        h["normalised1"] = ang3/baselineh
        h["normalised2"] = ang3 - baselineh
    
    f = pd.concat([f,g,h])
    return f

def generate_files_dlc(path=r"C:/MoitaLab",titles='Please select a directory',graphs = 'no',anglez = 'yes',frame_rate = 120):
    
    #loads the files, cuts them to the desired part, saves the leg positions and the joint angles in a new csv
    
    root = tk.Tk()
    dirname = fd.askdirectory(parent=root,initialdir=path,title=titles)
    root.destroy()
    
    #dirname = "C:/Users/Anna/Documents/Science/MoitaLab/Codes/Xana/tracking_csvs/walking_test"
    output_angles = pd.DataFrame(columns = ["filename","value","angle"])
    
    for direct, folders, files in os.walk(dirname):
    
            filenamez1 = glob.glob(direct + '/*' + '.csv')

            if len(filenamez1) > 0:
                for filez in filenamez1:
                    #print(filez)
                    #print(dirname)
                    
                    if "resnet" in filez:
                        print("")
                        print(filez)
                        ##import the csv with the mean values in it in Pandas
                        dataz = pd.read_csv(filez,skiprows = [0])
                        dataz = dataz.drop(0,axis="index") # drop the first row
                        dataz = dataz.astype(float)
                        dataz = dataz.apply(lambda x: x*-1 if ".1" in x.name else x) #invert all the y values (i.e those that contain ".1" in their name)
                        
                        #normalise the datapoints to the position of the thorax
                        mean_x = dataz.thorax.mean()
                        mean_y = dataz["thorax.1"].mean()    
                        dataz = dataz.apply(lambda y: y-mean_y if ".1" in y.name else y)
                        dataz = dataz.apply(lambda x: x-mean_x if not ".1" in x.name else x)
                        
                        #reduce the dataframe for efficiency of troubleshooting
                        #dataz = dataz.iloc[0:10]
                        
                        #print(dataz.columns.tolist)
                        #print(dataz)
                        namez = filez.split("\\")
                        namez2=namez[-1].split("DLC") #find the part of the filename before the DLC
                        namez2 = namez2[0][:-1]
                        print(namez2)
                        
                        #find the corresponding LED level csv
                        res = [ele for ele in filenamez1 if namez2 in ele] #find all the csvs that include the filename
                        
                        if "walking" in filez: #if we are doing a walk bout:
                            print("walking")
                            times = np.linspace(0,dataz.shape[0]/frame_rate,dataz.shape[0]) #make a list of times
                            dataz["Timepoint"] = times
                            print(times)
                            
                            
                            if anglez == 'yes':
                                if not any(x for x in res if "angles" in x):
                                    for eac in res:
                                        if "walkbouts" in eac:
                                            print(eac)
                                            bouts = pd.read_csv(eac)
                                            for bout in range(bouts.shape[0]):
                                                legz = dataz.iloc[bouts.iloc[bout].start-1:bouts.iloc[bout].end]
                                                new_filename = dirname + "//" + namez2 + "_" + str(bouts.iloc[bout].start-1) + "-" + str(bouts.iloc[bout].end-1)
                                                angles = leg_angles(legz,new_filename)
                                                #print(angles)
                                                angles.to_csv(new_filename+"_leg_angles.csv")
                                                
                                                legz_save = legz[["thorax","thorax.1","T1-coxa-femur","T1-coxa-femur.1","T1-femur-tibia","T1-femur-tibia.1","T1-tibia-tarsus","T1-tibia-tarsus.1","T1-claw","T1-claw.1","T2-claw","T2-claw.1","T3-claw","T3-claw.1","Timepoint"]]
                                                #print(dataz_save)
                                                legz_save.to_csv(new_filename+"_reduced_tracking.csv") 

                                
                                else:
                                    print("angles exists")
                                    
                                   
                        else:
                        
                            if "cropped" in filez: #if the file has been cropped to the 5s before the stim:
                                stim = 5*frame_rate #set the stim onset to 5s after the beginning of the video
                                #print(dataz.shape)
                                stim_times = np.linspace(0,(dataz.shape[0]-1)/frame_rate,dataz.shape[0]) #make a list of times CHECK THIS IS THE RIGHT SHAPE
                                stim_times = stim_times-stim_times[stim] #convert to times relative to stimulus 
                            
                            else:   
                                for eac in res:
                                    if eac != filez:
                                        if not "reduced_tracking" in eac:
                                            if not "leg_angles" in eac:
                                                LED = pd.read_csv(eac,header=None)#,skiprows = [0])
                                                print(LED.shape)
                                                if LED.shape[1] == 1: #if there is only the stim time and not the timestamp (early experiments)
                                                    stim = LED.idxmax() #find the moment the LED value increases
                                                    stim_times = np.linspace(0,LED.shape[0]/frame_rate,LED.shape[0]) #make a list of times
                                                    stim_times = stim_times-stim_times[stim] #convert to times relative to stimulus
                                                else:
                                                    LED.columns = ["LEDs","Timestamp"] #need to set the column names, then extract only the stim. Also make a graph of the frame rate so can check it for every fly.
                                                    stim = LED.LEDs.idxmax() #find the moment the LED value increases
                                                    stim_times = np.linspace(0,LED.shape[0]/frame_rate,LED.shape[0]) #make a list of times
                                                    stim_times = stim_times-stim_times[stim] #convert to times relative to stimulus
                                                    #print(LED)

                            dataz["Timepoint"] = stim_times
                    
                            #only choose the Xs before the stimulation time and X after the stimulation time
                            dataz = dataz.query("Timepoint > -5 and Timepoint < 25")
                        
                            if anglez == 'yes':
                                if not any(x for x in res if "angles" in x):
                                    angles = leg_angles(dataz,filez)
                                    angles.to_csv(dirname + "//" + namez2 + "_leg_angles.csv")
                                else:
                                    print("angles exists")
                            
                            dataz_save = dataz[["thorax","thorax.1","T1-coxa-femur","T1-coxa-femur.1","T1-femur-tibia","T1-femur-tibia.1","T1-tibia-tarsus","T1-tibia-tarsus.1","T1-claw","T1-claw.1","T2-claw","T2-claw.1","T3-claw","T3-claw.1","Timepoint"]]
                            #print(dataz_save)
                            dataz_save.to_csv(dirname + "//" + namez2 + "_reduced_tracking.csv") 
                        
                        g = dataz.shape[0]-1
                        
                        
                        
                        if graphs == 'yes':
                            fig, ax = plt.subplots(figsize = [6,4])
                            sn = sns.scatterplot(data=dataz,x="head",y="head.1",hue="bodyparts",palette="winter")
                            sn = sns.scatterplot(data=dataz,x="T1-claw",y="T1-claw.1",hue="bodyparts",palette="copper")
                            sn = sns.scatterplot(data=dataz,x="T2-claw",y="T2-claw.1",hue="bodyparts",palette="winter")
                            sn = sns.scatterplot(data=dataz,x="T3-claw",y="T3-claw.1",hue="bodyparts",palette="autumn")
                            plt.scatter(np.mean(dataz["T1-claw"]),np.mean(dataz["T1-claw.1"]))
                            plt.legend([],[], frameon=False)
                            plt.title("Positions of tarsus end for T1, T2 and T3")
                        
                        
                            mapz1 = []
                            mapz2 = []
                            mapz3 = []
                            for i in range(g):
                                col1 = [i/g,i/(2*g),1-i/g]
                                col2 = [1-i/g,i/g,i/g]
                                col3 = [i/(g),1-i/g,i/g]
                                mapz1.append(col1)
                                mapz2.append(col2)
                                mapz3.append(col3)
                            
                            fig, ax = plt.subplots(figsize = [6,4])
                            for i in range(g):
                                plt.plot([np.array(dataz["T1-claw"])[i],np.array(dataz["T1-claw"])[i+1]],[np.array(dataz["T1-claw.1"])[i],np.array(dataz["T1-claw.1"])[i+1]],c = mapz1[i])
                                plt.plot([np.array(dataz["T2-claw"])[i],np.array(dataz["T2-claw"])[i+1]],[np.array(dataz["T2-claw.1"])[i],np.array(dataz["T2-claw.1"])[i+1]],c = mapz2[i])
                                plt.plot([np.array(dataz["T3-claw"])[i],np.array(dataz["T3-claw"])[i+1]],[np.array(dataz["T3-claw.1"])[i],np.array(dataz["T3-claw.1"])[i+1]],c = mapz3[i])
                            plt.title("")

def load_files_DLC(dirname = "",path=r"C:/MoitaLab",titles='Please select a directory'):
    if len(dirname) == 0:
        root = tk.Tk()
        dirname = fd.askdirectory(parent=root,initialdir=path,title=titles)
        root.destroy()
    output_angles = pd.DataFrame(columns = ["value","angle","timepoints","filename"])
    bodyparts = pd.DataFrame(columns = ["Timepoint","Bodypart","value","Filename"])
    
    for direct, folders, files in os.walk(dirname):
        filenamez1 = glob.glob(direct + '/*' + '.csv')
        if len(filenamez1) > 0:
            for filez in filenamez1:
                if "leg_angles" in filez:
                    angle = pd.read_csv(filez,index_col = [0])
                    #print(angle)
                    output_angles = pd.concat([output_angles,angle])
                    #print(output_angles)
                    #print(output_angles.shape)
    
                elif "reduced_tracking" in filez:
                    data = pd.read_csv(filez,index_col = [0])
                    #print(data)
                    data2 = pd.melt(data,id_vars = ["Timepoint"],var_name = "Bodypart")
                    data2["Filename"] = filez
                    bodyparts = pd.concat([bodyparts,data2])
                    
    return(output_angles,bodyparts)
                    

## Generate the angles and bodyparts files from the raw data

In [ ]:
#generate_files_dlc(graphs = 'no',anglez='yes')

## Load the angles and bodyparts files

In [ ]:
output_angles,bodyparts = load_files_DLC(dirname = "C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/welltracked_stim/rest",titles = "Please select stimulated files")

In [ ]:
angles_walking, bodyparts_walking = load_files_DLC(dirname = "C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/welltracked_walking",titles = "Please select walking files")

In [ ]:
#round the files, otherwise sns.lineplot has a bug (can't cope with a lot of decimal places)
output_angles_round = output_angles.round(2)
bodyparts_round = bodyparts.round(2)
#angles_walking = angles_walking.round(3)
bodyparts_walking_round = bodyparts_walking.round(2)



In [ ]:
#print(angles_walking)
good_tracking = pd.read_excel('C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/good_tracking_periods.xlsx')

In [ ]:
#print(good_tracking)

## Separate the intact and decapitated flies

In [ ]:
decapz = output_angles.query("filename.str.contains('decap')",engine = "python")
intact = output_angles.merge(decapz.drop_duplicates(), how='left', indicator=True)
intact = intact.query("_merge == 'left_only'")
intact_ATR = intact.query("filename.str.contains('wAT')",engine = "python")


decapz_body = bodyparts.query("Filename.str.contains('decap')",engine = "python")
intact_body = bodyparts.merge(decapz_body.drop_duplicates(), how='left', indicator=True)
intact_body = intact_body.query("_merge == 'left_only'")
intact_body_ATR = intact_body.query("Filename.str.contains('wAT')",engine = "python")

In [ ]:
intact_ATR_round = intact_ATR.round(2)
decapz_round = decapz.round(2)

### Add the relative time to angles walking

In [ ]:
print(angles_walking.shape)

In [ ]:
timez = np.array([])
for file in angles_walking.filename.unique():
    #print(file)
    #print(angles_walking.query("filename == @file"))
    times = np.linspace(0,(angles_walking.query('filename == @file and angle == 1').timepoints.max()-angles_walking.query('filename == @file and angle == 1').timepoints.min()),angles_walking.query('filename == @file and angle == 1').timepoints.shape[0])
    #print(times.shape)
    for ang in range(3):
        timez = np.hstack([timez,times])
    #print(timez.shape)
    #print("")
print(timez.shape)

In [ ]:
angles_walking["Relative_times"] = timez
angles_walking_round = angles_walking.round(3)

In [ ]:
print(angles_walking)

## Joint angles 

### Walking

In [ ]:
print(angles_walking.filename.unique().shape)

cmapper = 'Oranges'

brg = cm.get_cmap(cmapper,4)
#print(angles_walking.filename.unique().shape[0])
cmapz = brg(range(4))

for file in angles_walking.filename.unique():
    #print(file)
    fig, ax = plt.subplots(figsize = [12,4])
    for i,ang in enumerate(angles): 
        k = i+1
        #times = np.linspace(0,(angles_walking.query('filename == @file and angle == @k').timepoints.max()-angles_walking.query('filename == @file and angle == @k').timepoints.min()),angles_walking.query('filename == @file and angle == @k').timepoints.shape[0])
        #print(times)
        #ax.plot(angles_walking.query('filename == @file and angle == @k').timepoints,angles_walking.query('filename == @file and angle == @k').value,label=ang)
        ax.plot(angles_walking.query('filename == @file and angle == @k').Relative_times,angles_walking.query('filename == @file and angle == @k').value,label=ang,c=cmapz[3-i])

    ax.set_title(file[50:])
    ax.set_xlabel("Time (s)",fontsize = 15)
    ax.set_ylabel("Joint angle (degrees)",fontsize = 15)
    #ax.legend()
    ax.set_xlim([0,4])
    
    #fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/walking_traces.svg")

In [ ]:
print(angles_walking.filename.unique().shape)
for file in angles_walking.filename.unique():
    #print(file)
    fig, ax = plt.subplots(figsize = [17,4])
    for i,ang in enumerate(angles): 
        k = i+1
        times = np.linspace(0,(angles_walking.query('filename == @file and angle == @k').timepoints.max()-angles_walking.query('filename == @file and angle == @k').timepoints.min()),angles_walking.query('filename == @file and angle == @k').timepoints.shape[0])
        #print(times)
        #ax.plot(angles_walking.query('filename == @file and angle == @k').timepoints,angles_walking.query('filename == @file and angle == @k').value,label=ang)
        ax.plot(times,angles_walking.query('filename == @file and angle == @k').value,label=ang,c=cmapz[3-i])

    #ax.set_title(file[50:])
    ax.set_xlabel("Time (s)",fontsize = 15)
    ax.set_ylabel("Joint angle (degrees)",fontsize = 15)
    ax.legend()
    #ax.set_xlim([])

### Intact

In [ ]:
for each in intact_ATR.filename.unique():
    fig, ax = plt.subplots(figsize = [17,4])
    g = intact_ATR.query('filename == @each')
    for i,ang in enumerate(angles): 
        k = i+1
        ax.plot(g.query('angle == @k').timepoints,g.query('angle == @k').value,label=ang,c=cmapz[3-i])
    ax.set_xlabel("Time from stim (s)",fontsize = 15)
    ax.set_ylabel("Joint angle (degrees)",fontsize = 15)
    ax.set_xlim([-0.5,5])
    ax.set_title(each[50:])
    #ax.legend()  
    
    #fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/stim_intact_traces.svg")

### Decapitated

In [ ]:
cmapper = 'Oranges'
brg = cm.get_cmap(cmapper,4)
cmapz = brg(range(4))
for l,each in enumerate(decapz.filename.unique()):
    #print(each)
    fig, ax = plt.subplots(figsize = [17,4])
    g = decapz.query('filename == @each')
    for i,ang in enumerate(angles): 
        k = i+1
        ax.plot(g.query('angle == @k').timepoints,g.query('angle == @k').value,label=ang,c=cmapz[3-i])
    #ax.set_title(each)
    ax.set_xlabel("Time from stim (s)",fontsize = 15)
    ax.set_ylabel("Joint angle (degrees)",fontsize = 15)
    ax.set_xlim([-0.5,5])
    ax.set_title(each[70:])
    #ax.legend()
    
    
    fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/decap_traces"+str(l)+".png")

## Remove parts after stimulation when not moving

### Intact flies

In [ ]:
stim_intact_moving = pd.DataFrame(columns=intact_ATR.columns)

for file in intact_ATR.filename.unique():
    #print(good_tracking.query("Filename == @file"))
    first_start = float(good_tracking.query("Filename == @file").first_start)
    first_end = float(good_tracking.query("Filename == @file").first_end)
    good = intact_ATR.query("filename == @file and timepoints > @first_start and timepoints < @first_end")
    stim_intact_moving = pd.concat([stim_intact_moving,good])
    #print(good.shape)
    
    second_start = float(good_tracking.query("Filename == @file").second_start)
    second_end = float(good_tracking.query("Filename == @file").second_end)
    if second_start > 0:
        #print(second_start)
        #print(second_end)
        good = intact_ATR.query("filename == @file and timepoints > @second_start and timepoints < @second_end")
        stim_intact_moving = pd.concat([stim_intact_moving,good])
        #print(good.shape)
        
#for file in stim_intact_moving.filename.unique():
#    print(file)
#    print(stim_intact_moving.query("filename == @file").timepoints)
#    print("")

#Bodyparts positions
stim_intact_moving_body = pd.DataFrame(columns=intact_body_ATR.columns)
for file in intact_body_ATR.Filename.unique():
    #print(file)
    reduced = file[-57:-22]
    #print(reduced)
    #print(good_tracking.query("Filename.str.contains(@reduced)",engine = "python").first_start)
    first_start = float(good_tracking.query("Filename.str.contains(@reduced)",engine = "python").first_start)
    #print(first_start)
    first_end = float(good_tracking.query("Filename.str.contains(@reduced)",engine = "python").first_end)
    #print(first_end)
    
    good = intact_body_ATR.query("Filename == @file and Timepoint > @first_start and Timepoint < @first_end")
    stim_intact_moving_body = pd.concat([stim_intact_moving_body,good])
    #print(good.shape)
    
    #print(good_tracking.query("Filename.str.contains(@reduced)",engine = "python").second_start)
    
    second_start = float(good_tracking.query("Filename.str.contains(@reduced)",engine = "python").second_start)
    second_end = float(good_tracking.query("Filename.str.contains(@reduced)",engine = "python").second_end)
    if second_start > 0:
        #print(second_start)
        #print(second_end)
        good = intact_body_ATR.query("Filename == @file and Timepoint > @second_start and Timepoint < @second_end")
        stim_intact_moving_body = pd.concat([stim_intact_moving_body,good])
        #print(good.shape)
    #print("")

#for file in stim_intact_moving_body.Filename.unique():
#    print(file)
#    print(stim_intact_moving_body.query("Filename == @file and Bodypart == 'thorax'").Timepoint)
#    print("")

### Decapitated

In [ ]:
stim_decap_moving = pd.DataFrame(columns=decapz.columns)

for file in decapz.filename.unique():
    #print(good_tracking.query("Filename == @file"))
    first_start = float(good_tracking.query("Filename == @file").first_start)
    first_end = float(good_tracking.query("Filename == @file").first_end)
    good = decapz.query("filename == @file and timepoints > @first_start and timepoints < @first_end")
    stim_decap_moving = pd.concat([stim_decap_moving,good])
    #print(good.shape)
    
    second_start = float(good_tracking.query("Filename == @file").second_start)
    second_end = float(good_tracking.query("Filename == @file").second_end)
    if second_start > 0:
        #print(second_start)
        #print(second_end)
        good = decapz.query("filename == @file and timepoints > @second_start and timepoints < @second_end")
        stim_decap_moving = pd.concat([stim_decap_moving,good])
        #print(good.shape)

stim_decap_moving_body = pd.DataFrame(columns=decapz_body.columns)
for file in decapz_body.Filename.unique():

    reduced = file.split("\\")[-1]
    red2 = reduced.split("reduced_tracking")[0][0:-8]

    #print("")
    
    first_start = float(good_tracking.query("Filename.str.contains(@red2)",engine = "python").first_start)
    first_end = float(good_tracking.query("Filename.str.contains(@red2)",engine = "python").first_end)
    
    good = decapz_body.query("Filename == @file and Timepoint > @first_start and Timepoint < @first_end")
    stim_decap_moving_body = pd.concat([stim_decap_moving_body,good])
    #print(good.shape)
    
    #print(good_tracking.query("Filename.str.contains(@reduced)",engine = "python").second_start)
    
    second_start = float(good_tracking.query("Filename.str.contains(@red2)",engine = "python").second_start)
    second_end = float(good_tracking.query("Filename.str.contains(@red2)",engine = "python").second_end)
    if second_start > 0:
        #print(second_start)
        #print(second_end)
        good = decapz_body.query("Filename == @file and Timepoint > @second_start and Timepoint < @second_end")
        stim_decap_moving_body = pd.concat([stim_decap_moving_body,good])
        #print(good.shape)
    #print("")       

In [ ]:
for each in stim_decap_moving.query("timepoints > 0 and timepoints < 1").filename.unique():
    print(each)

## Average plots

In [ ]:
fig, ax = plt.subplots(figsize = [15,8])
sns.lineplot(data = angles_walking_round.query("angle == 1"),x='Relative_times',y='value',hue='filename')#,palette = 'viridis')
ax.set_xlim([0,1])
#fig, ax = plt.subplots(figsize = [10,4])#
#sns.lineplot(data = intact.query("filename.str.contains(@cond) and angle == 1",engine = "python"),x='timepoints',y='value',hue='filename',palette = 'viridis')
#ax.set_xlim([-2,4])
#fig, ax = plt.subplots(figsize = [10,6])
#sns.lineplot(data = intact.query("filename.str.contains(@cond)",engine = "python"),x='timepoints',y='value',hue='angle',palette = 'viridis')
#sns.lineplot(data = intact.query("filename.str.contains(@cond)",engine = "python"),x='timepoints',y='value',hue='angle',palette = 'viridis')
#fig, ax = plt.subplots(figsize = [10,4])
#sns.lineplot(data = decapz.query("filename.str.contains(@cond)",engine = "python"),x='timepoints',y='value',hue='angle',palette = 'viridis')
#ax.set_xlim([-2,4])

In [ ]:
fig, ax = plt.subplots(figsize = [15,8])
sns.lineplot(data = intact_ATR_round.query("timepoints > -2 and timepoints < 5"),x='timepoints',y='value',hue='angle',palette = 'tab10')
ax.set_xlim([-0.5,5])
#fig, ax = plt.subplots(figsize = [10,4])#
#sns.lineplot(data = intact.query("filename.str.contains(@cond) and angle == 1",engine = "python"),x='timepoints',y='value',hue='filename',palette = 'viridis')
#ax.set_xlim([-2,4])
#fig, ax = plt.subplots(figsize = [10,6])
#sns.lineplot(data = intact.query("filename.str.contains(@cond)",engine = "python"),x='timepoints',y='value',hue='angle',palette = 'viridis')
#sns.lineplot(data = intact.query("filename.str.contains(@cond)",engine = "python"),x='timepoints',y='value',hue='angle',palette = 'viridis')
#fig, ax = plt.subplots(figsize = [10,4])
#sns.lineplot(data = decapz.query("filename.str.contains(@cond)",engine = "python"),x='timepoints',y='value',hue='angle',palette = 'viridis')
#ax.set_xlim([-2,4])

In [ ]:
fig, ax = plt.subplots(figsize = [15,8])
sns.lineplot(data = decapz_round,x='timepoints',y='value',hue='angle',palette = 'tab10')
ax.set_xlim([-0.5,5])
#fig, ax = plt.subplots(figsize = [10,4])#
#sns.lineplot(data = intact.query("filename.str.contains(@cond) and angle == 1",engine = "python"),x='timepoints',y='value',hue='filename',palette = 'viridis')
#ax.set_xlim([-2,4])
#fig, ax = plt.subplots(figsize = [10,6])
#sns.lineplot(data = intact.query("filename.str.contains(@cond)",engine = "python"),x='timepoints',y='value',hue='angle',palette = 'viridis')
#sns.lineplot(data = intact.query("filename.str.contains(@cond)",engine = "python"),x='timepoints',y='value',hue='angle',palette = 'viridis')
#fig, ax = plt.subplots(figsize = [10,4])
#sns.lineplot(data = decapz.query("filename.str.contains(@cond)",engine = "python"),x='timepoints',y='value',hue='angle',palette = 'viridis')
#ax.set_xlim([-2,4])

## Short Time Fourier transform

In [ ]:
def FFT_angles(data,freq_cutoff_low = 0):
    Zxx_base=pd.DataFrame(columns = ["Filename","angle","Hz","Power","Fly"]) #power of the signal
    t_base=[] 
    
    for ang in np.arange(1,4):
    #print(ang)
    
        Zxx_base_fly = []
        Hz_base = []
        
        for file in data.filename.unique():
            data_test = data.query('filename == @file and angle == @ang').value
            #normalise the value to centre it on 0 by subtracting the mean value
            data_test = data_test-np.mean(data_test)   
                
            Hz, t, Zxx1 = scisig.stft(data_test, frame_rate, window='hamming', nfft=256, nperseg = 40)#,noverlap=1)
            
            data_smoothed = data_test.rolling(3,center = True).mean()
            Hz_s, t_s, Zxx1_s = scisig.stft(data_smoothed, frame_rate, window='hamming', nfft=512, nperseg = 40)#,noverlap=39)
            
            #print(Hz)
            #print(t)
            #print(Zxx1)
            to_add = pd.DataFrame(np.nanmean(np.abs(Zxx1),axis=1),columns = ["Power"])
            to_add["Hz"] = Hz
            to_add["Filename"] = file
            to_add["angle"] = ang
            fly = file.find("Fly")
            fly = file[fly:fly+4]
            to_add["Fly"] = fly
            #print(to_add)
            
            Zxx_base = pd.concat([Zxx_base,to_add])
            
            #fig,ax = plt.subplots(figsize=([10,4]))
            #ax.pcolormesh(t_s, Hz_s[Hz_s > freq_cutoff_low], np.abs(Zxx1_s)[Hz_s > freq_cutoff_low], vmin=0, shading='gouraud')
            #ax.set_title(str('STFT Magnitude_smoothed, ang ' + str(ang)))
            #ax.set_ylim([freq_cutoff_low,60])
            
            fig,ax = plt.subplots(figsize=([10,4]))
            ax.pcolormesh(t, Hz[Hz > freq_cutoff_low], np.abs(Zxx1)[Hz > freq_cutoff_low], vmin=0, shading='gouraud')
            ax.set_title(str('STFT Magnitude, ang ' + str(ang)))
            ax.set_xlabel("Time (s)",fontsize = 12)
            ax.set_ylabel("Hz",fontsize = 12)
            #ax.set_ylim([freq_cutoff_low,20])
            
            
            #fig,ax = plt.subplots(figsize=([15,6]))
            #for i in range(6):
            #    data_smooth = data_test.rolling(i+1,center = True).mean()
            #    Hz, t, Zxx1 = scisig.stft(data_smooth, 1/desired_framerate, window='hamming', nfft=256, nperseg = 60,noverlap=59)
            #    plt.plot(Hz[Hz > freq_cutoff_low],np.nanmean(np.abs(Zxx1[Hz > freq_cutoff_low]),axis=1),label = str(i+1))
            #plt.legend()
               #
               
                #plt.plot(Hz[Hz > freq_cutoff_low],np.nanmean(np.abs(Zxx1[Hz > freq_cutoff_low]),axis=1))
                #plt.plot(Hz[Hz > freq_cutoff_low],np.nanmean(np.abs(Zxx1_s[Hz_s > freq_cutoff_low]),axis=1))
                #plt.show
                
    return(Zxx_base)       

In [ ]:
Zxx_base_walking = FFT_angles(angles_walking)
Zxx_base_stim = FFT_angles(stim_intact_moving.query("timepoints > 0 and timepoints < 1"))
Zxx_base_stim_later = FFT_angles(stim_intact_moving.query("timepoints > 3 and timepoints < 5"))
Zxx_base_decap = FFT_angles(stim_decap_moving.query("timepoints > 0 and timepoints < 1"))

In [ ]:
print(pd.pivot_table(Zxx_base_walking.query("angle==1"),index = "Hz"))
print("")
print(pd.pivot_table(Zxx_base_stim.query("angle == 1"),index = "Hz"))
print("")
print(pd.pivot_table(Zxx_base_stim_later.query("angle == 1"),index = "Hz"))
print("")
print(pd.pivot_table(Zxx_base_decap.query("angle == 1"),index = "Hz"))


#stancez_mean = pd.pivot_table(data = stancez, index = 'bin', values = 'Angle2')

In [ ]:
fig,ax = plt.subplots(figsize=([14,8]))
sn = sns.lineplot(x="Hz",y="Power",data=Zxx_base_walking.query("angle == 1"),color = walk,label = "Walking")
sn = sns.lineplot(x="Hz",y="Power",data=Zxx_base_stim.query("angle == 1"),color = si,label = "Stim early")
sn = sns.lineplot(x="Hz",y="Power",data=Zxx_base_stim_later.query("angle == 1"),color = sl,label = "Stim late")
sn = sns.lineplot(x="Hz",y="Power",data=Zxx_base_decap.query("angle == 1"),color = dec,label = "Decap stim")
plt.xlim([-1,20])
plt.ylim([0,15])
plt.legend(fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Coxa-femur",fontsize=20)

#ax.grid(False)


fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/fft.svg")


In [ ]:
cmapper = 'Oranges'

brg = cm.get_cmap(cmapper,4)
#print(angles_walking.filename.unique().shape[0])
cmapz = brg(range(4))
#print(cmapz)

fig,ax = plt.subplots(figsize=([10,5]))
for ang in np.arange(1,4):
    sn = sns.lineplot(x="Hz",y="Power",data=Zxx_base_walking.query("angle == @ang"),label=angles[ang-1],c=cmapz[4-ang])
plt.xlim([-1,20])
plt.ylim([0,25])
plt.legend(fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Walking",fontsize = 20)
#fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/fft-supp-walking-25.svg")

In [ ]:
fig,ax = plt.subplots(figsize=([10,5]))
for ang in np.arange(1,4):
    sn = sns.lineplot(x="Hz",y="Power",data=Zxx_base_stim.query("angle == @ang"),label=angles[ang-1],c=cmapz[4-ang])#,hue="angle")#,palette="brg")
plt.xlim([-1,20])
plt.ylim([0,25])
plt.legend(fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Stim Initial",fontsize = 20)
#fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/fft-supp-stim-initial-25.svg")

In [ ]:
fig,ax = plt.subplots(figsize=([10,5]))
for ang in np.arange(1,4):
    sn = sns.lineplot(x="Hz",y="Power",data=Zxx_base_stim_later.query("angle == @ang"),label=angles[ang-1],c=cmapz[4-ang])#,hue="angle")#,palette="brg")
plt.xlim([-1,20])
plt.ylim([0,25])
plt.legend(fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Stim Late",fontsize = 20)
#fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/fft-supp-stim-late-25.svg")

In [ ]:
fig,ax = plt.subplots(figsize=([10,5]))
for ang in np.arange(1,4):
    sn = sns.lineplot(x="Hz",y="Power",data=Zxx_base_decap.query("angle == @ang"),label=angles[ang-1],c=cmapz[4-ang])#,hue="angle")#,palette="brg")
plt.xlim([-1,20])
plt.ylim([0,20])
plt.legend(fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Decapitated",fontsize = 20)
fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/fft-supp-decap.svg")

In [ ]:
fig,ax = plt.subplots(figsize=([10,5]))
sn = sns.lineplot(x="Hz",y="Power",data=Zxx_base_decap.query("angle == 1"),hue="Filename")
plt.xlim([0,20])
#plt.ylim([0,20])
plt.title("Angle 1")
plt.legend("")

fig,ax = plt.subplots(figsize=([10,5]))
sn = sns.lineplot(x="Hz",y="Power",data=Zxx_base_decap.query("angle == 2"),hue="Filename")
plt.xlim([0,20])
#plt.ylim([0,20])
plt.title("Angle 2")
plt.legend()


fig,ax = plt.subplots(figsize=([10,5]))
sn = sns.lineplot(x="Hz",y="Power",data=Zxx_base_decap.query("angle == 3"),hue="Filename")
plt.xlim([0,20])
#plt.ylim([0,20])
plt.title("Angle 3")
plt.legend("")



## Autocorrelations of signals

In [ ]:
def acorrs_between_signals(data,joint,max_lags = 100,name = ""): #nb if using stim data and only want to do this for the baseline, need to specify this when inputting the data
    acorr_all = []
    intervals_all = []
    for file in data.filename.unique():
        test_data = data.query('filename == @file and angle==@joint').value
        test_data = test_data-np.mean(test_data)
        test_data.reset_index()
        
        fig,ax1 = plt.subplots(figsize=(10,5))
         
        acor = plt.acorr(test_data,maxlags = test_data.shape[0]-1)
        plt.title("autocorrelation"+file)
        
        times = np.linspace(acor[0].min()/frame_rate,acor[0].max()/frame_rate,acor[0].shape[0])
        times = times.round(4)
        #print(times)
                          
        if len(acorr_all) == 0:
            lags = acor[0]
            acorr_all = pd.DataFrame(acor[1],columns = ["Value"])
            acorr_all["file"] = file
            acorr_all["Condition"] = name
            acorr_all["lags"] = acor[0]
            acorr_all["times"] = times
                #    intervals_all = intervals
        #    #print(acorr_all.shape)
        else:
            to_add = pd.DataFrame(acor[1],columns = ["Value"])
            to_add["file"] = file
            to_add["Condition"] = name
            to_add["lags"] = acor[0]
            to_add["times"] = times
            acorr_all = pd.concat([acorr_all,to_add])
    
    return(acorr_all)#,lags,intervals_all,times)

In [ ]:
acorr_walking = acorrs_between_signals(angles_walking,1,name = "Walking")
acorr_stim_initial = acorrs_between_signals(stim_intact_moving.query("timepoints > 0 and timepoints < 1"),1,name = "Stim Initial",max_lags = 76)
acorr_stim_late = acorrs_between_signals(stim_intact_moving.query("timepoints > 3 and timepoints < 5"),1,name = "Stim late")
acorr_stim_decap = acorrs_between_signals(stim_decap_moving.query("timepoints > 0 and timepoints < 1"),1,name = "Decapitated")

In [ ]:
for filez in acorr_walking.file.unique()[3:4]:
    fig,ax1 = plt.subplots(figsize=(11,5))
    ax1.plot(acorr_walking.query("file == @filez").times,acorr_walking.query("file == @filez").Value,c='g',linewidth = 4)
    ax1.set_xlim([-0.5,0.5])
    ax1.set_ylim([-0.5,1.1])
    ax1.set_title("Walking",fontsize = 20)
    ax1.set_xlabel("Time")
    
for filez in acorr_stim_initial.file.unique()[1:2]:
    fig,ax1 = plt.subplots(figsize=(11,5))
    ax1.plot(acorr_stim_initial.query("file == @filez").times,acorr_stim_initial.query("file == @filez").Value,c='r',linewidth = 4)
    ax1.set_xlim([-0.5,0.5])
    ax1.set_ylim([-0.5,1.1])
    ax1.set_title("Stim Initial",fontsize = 20)
    ax1.set_xlabel("Time")

for filez in acorr_stim_late.file.unique()[2:3]:
    fig,ax1 = plt.subplots(figsize=(11,5))
    ax1.plot(acorr_stim_late.query("file == @filez").times,acorr_stim_late.query("file == @filez").Value,c='k',linewidth = 4)
    ax1.set_xlim([-0.5,0.5])
    ax1.set_ylim([-0.5,1.1])
    ax1.set_title("Stim Late",fontsize = 20)
    ax1.set_xlabel("Time")

for filez in acorr_stim_decap.file.unique()[5:6]:
    fig,ax1 = plt.subplots(figsize=(11,5))
    ax1.plot(acorr_stim_decap.query("file == @filez").times,acorr_stim_decap.query("file == @filez").Value,c='b',linewidth = 4)
    ax1.set_xlim([-0.5,0.5])
    ax1.set_ylim([-0.5,1.1])
    ax1.set_title("Decapitated",fontsize = 20)
    ax1.set_xlabel("Time")

In [ ]:
acorrs_all = pd.concat([acorr_walking,acorr_stim_initial,acorr_stim_late,acorr_stim_decap])

fig,ax1 = plt.subplots(figsize=(11,5))
sns.lineplot(data=acorrs_all,x="times",y="Value",hue="Condition")
ax1.set_xlim([-0.3,0.3])

## Angle-angle plots

In [ ]:
def joint_angle_line(data,upp,low,col = 'k',ang1=1,ang2=2,alph = 0.2,**ax_n):
    #print(len(ax_n))
    for i in range(data.query('angle == 1 and timepoints > @low and timepoints < @upp').shape[0]-1):
        e = data.query('angle == @ang1 and timepoints > @low and timepoints < @upp').value.iloc[i]
        f = data.query('angle == @ang1 and timepoints > @low and timepoints < @upp').value.iloc[i+1]
        g = data.query('angle == @ang2 and timepoints > @low and timepoints < @upp').value.iloc[i]
        h = data.query('angle == @ang2 and timepoints > @low and timepoints < @upp').value.iloc[i+1]
        if len(ax_n)<1:
            ax.plot([e,f],[g,h],linewidth = 2,c=col,alpha=alph)
        elif len(ax_n) == 1:
            ax[ax_n["ax_n1"]].plot([e,f],[g,h],linewidth = 2,c=col,alpha=alph)
        else:
            ax[ax_n["ax_n1"],ax_n["ax_n2"]].plot([e,f],[g,h],linewidth = 2,c=col,alpha=alph)

bins1 = np.arange(0,130,5)
bins2 = np.arange(0,185,5)
bins3 = np.arange(60,190,5)

In [ ]:
## NB trajectories travel CLOCKWISE around the 'doughnut'

### Coxa-Femur and Femur-Tibia

In [ ]:
cmapper = 'gist_ncar'

brg = cm.get_cmap(cmapper,angles_walking.filename.unique().shape[0])
#print(angles_walking.filename.unique().shape[0])
cmapz = brg(range(angles_walking.filename.unique().shape[0]))
fig, ax = plt.subplots(1,4,figsize = [22,5])

for each, i in enumerate(angles_walking.filename.unique()): #nb only use the first 1s here for the line plot to be in line with the 1s used in the stimulated cases
    joint_angle_line(angles_walking.query('filename == @i'),angles_walking.query('filename == @i').timepoints.max(),angles_walking.query('filename == @i').timepoints.min(),col = cmapz[each],alph=1/(angles_walking.shape[0])*700,ax_n1=0)   
        
ax[0].set_title("Joint angles while walking ")
ax[0].set_xlabel("Coxa-Femur angle",fontsize = 12)
ax[0].set_ylabel("Femur-Tibia angle",fontsize = 12)
ax[0].set_xlim([-20,160])
ax[0].set_ylim([0,180])



brg = cm.get_cmap(cmapper,stim_intact_moving.filename.unique().shape[0]+1)
#print(stim_intact_moving.filename.unique().shape[0])
cmapz = brg(range(stim_intact_moving.filename.unique().shape[0]+1))
#fig, ax = plt.subplots(1,2,figsize = [12,5])

for each, i in enumerate(stim_intact_moving.filename.unique()):
    joint_angle_line(stim_intact_moving.query('filename == @i'),1,0,col = cmapz[each],ax_n1=1)
ax[1].set_title("Joint angles after stimulation (first 1 s)")
ax[1].set_xlabel("Coxa-Femur angle",fontsize = 12)
ax[1].set_ylabel("Femur-Tibia angle",fontsize = 12)
ax[1].set_xlim([-20,160])
ax[1].set_ylim([0,180])

brg = cm.get_cmap(cmapper,stim_intact_moving.query('timepoints > 3').filename.unique().shape[0]+1)
#print(stim_intact_moving.filename.unique().shape[0])
cmapz = brg(range(stim_intact_moving.query('timepoints > 3').filename.unique().shape[0]+1))

#fig, ax = plt.subplots(1,2,figsize = [12,5])
for each, i in enumerate(stim_intact_moving.query('timepoints > 3').filename.unique()):
    joint_angle_line(stim_intact_moving.query('filename == @i'),5,3,col = cmapz[each],ax_n1 = 2)
ax[2].set_title("Joint angles after stimulation (3-5s)")
ax[2].set_xlabel("Coxa-Femur angle",fontsize = 12)
ax[2].set_ylabel("Femur-Tibia angle",fontsize = 12)
ax[2].set_xlim([-20,160])
ax[2].set_ylim([0,180])

#
#
#
brg = cm.get_cmap(cmapper,stim_decap_moving.filename.unique().shape[0]+1)
cmapz = brg(range(stim_decap_moving.filename.unique().shape[0]+1))
#fig, ax = plt.subplots(1,2,figsize = [12,5])

for each, i in enumerate(stim_decap_moving.filename.unique()):
    joint_angle_line(stim_decap_moving.query('filename == @i'),1,0,col = cmapz[each],ax_n1=3)
ax[3].set_title("Joint angles after stimulation decapitated (first 1 s)")
ax[3].set_xlabel("Coxa-Femur angle",fontsize = 12)
ax[3].set_ylabel("Femur-Tibia angle",fontsize = 12)
ax[3].set_xlim([-20,160])
ax[3].set_ylim([0,180])

fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/angle-angleplots.svg")

# Differentiate traces

In [ ]:
def diff_traces(data,graphs = 'no'):
    difz_all = pd.DataFrame(columns = ["value","angle","filename","timepoints"])
    for each, file in enumerate(data.filename.unique()):
        if graphs == 'yes':
            fig, ax = plt.subplots(figsize = [15,4])
            fig2, ax2 = plt.subplots(figsize = [15,4])
        
        for i,ang in enumerate(angles): 
            k = i+1
            difz = data.query('angle == @k and filename == @file').value.diff()
            difz2 = pd.DataFrame(difz,columns = ["value"])
            difz2["angle"] = i
            difz2["filename"] = file
            difz2["timepoints"] = data.query('angle == @k and filename == @file').timepoints
            
            difz_all = pd.concat([difz_all,difz2])
        
            if graphs == 'yes':
                ax.plot(angles_walking.query('angle == @k and filename == @file').timepoints,angles_walking.query('angle == @k and filename == @file').value,label=ang)#,c=cmapz[3-i])
                ax2.plot(angles_walking.query('angle == @k and filename == @file').timepoints,difz,label=ang)#,c=cmapz[3-i])
        
    return(difz_all)

In [ ]:
diff_walk = diff_traces(angles_walking,graphs = 'yes')

In [ ]:
fig, ax = plt.subplots(figsize = [15,15])
for each in diff_walk.filename.unique():
    #ax.plot(diff_walk.query("filename == @each and angle == 1 and timepoints < 1 and timepoints > 0").value,stim_intact_moving.query("filename == @each and angle == 2 and timepoints < 1 and timepoints > 0").value,c = cmapz[j],linewidth = 0.5,alpha = 0.5)
    ax.scatter(diff_walk.query("filename == @each and angle == 1").value,diff_walk.query("filename == @each and angle == 2").value,linewidth = 0.1,s=10)
    #ax[0].scatter(merged.query("phase == @k and angle == 1 and cycle == @cyc").value,merged.query("phase == @k and angle == 2 and cycle == @cyc").value,s=1,alpha=0.5)
    
fig, ax = plt.subplots(figsize = [8,6])
ax.scatter(diff_walk.query("angle == 1").value,diff_walk.query("angle == 2").value,linewidth = 0.1,s=10)
    #ax[0].scatter(merged.query("phase == @k and angle == 1 and cycle == @cyc").value,merged.query("phase == @k and angle == 2 and cycle == @cyc").value,s=1,alpha=0.5)
    


In [ ]:
#print(stim_intact_moving.query('timepoints > 3'))
#print(stim_intact_moving.query('timepoints > 3'))
difz_walk = pd.DataFrame(columns = ["value","angle","filename","timepoints"])
for each, file in enumerate(angles_walking.filename.unique()):
    fig, ax = plt.subplots(figsize = [15,4])
    fig2, ax2 = plt.subplots(figsize = [15,4])
    for i,ang in enumerate(angles): 
        k = i+1
        difz = angles_walking.query('angle == @k and filename == @file').value.diff()
        difz2 = pd.DataFrame(difz,columns = ["value"])
        difz2["angle"] = i
        difz2["filename"] = file
        difz2["timepoints"] = angles_walking.query('angle == @k and filename == @file').timepoints
        
        difz_walk = pd.concat([difz_walk,difz2])
print(difz_walk)
        #print(stim_intact_moving.query('angle == @k and timepoints > 3 and filename == @file').timepoints)
        
        #ax.plot(angles_walking.query('angle == @k and filename == @file').timepoints,angles_walking.query('angle == @k and filename == @file').value,label=ang)#,c=cmapz[3-i])
        
        #ax2.plot(angles_walking.query('angle == @k and filename == @file').timepoints,difz,label=ang)#,c=cmapz[3-i])
        




difz_walk = pd.DataFrame(columns = ["value","angle","filename","timepoints"])
for each, file in enumerate(stim_intact_moving.query('timepoints > 0 and timepoints < 1').filename.unique()):
    fig, ax = plt.subplots(figsize = [15,4])
    fig2, ax2 = plt.subplots(figsize = [15,4])
    for i,ang in enumerate(angles): 
        k = i+1
        difz = stim_intact_moving.query('angle == @k and timepoints > 0 and timepoints < 1 and filename == @file').value.diff()

        #print(stim_intact_moving.query('angle == @k and timepoints > 3 and filename == @file').timepoints)
        ax.plot(stim_intact_moving.query('angle == @k and timepoints > 0 and timepoints < 1 and filename == @file').timepoints,stim_intact_moving.query('angle == @k and timepoints > 0 and timepoints < 1 and filename == @file').value,label=ang)#,c=cmapz[3-i])
        
        ax2.plot(stim_intact_moving.query('angle == @k and timepoints > 0 and timepoints < 1 and filename == @file').timepoints,difz,label=ang)#,c=cmapz[3-i])
        
        

for each, file in enumerate(stim_intact_moving.query('timepoints > 3').filename.unique()):
    fig, ax = plt.subplots(figsize = [10,4])
    fig2, ax2 = plt.subplots(figsize = [15,4])
    for i,ang in enumerate(angles): 
        k = i+1
        #print(stim_intact_moving.query('angle == @k and timepoints > 3 and filename == @file').timepoints)
        ax.plot(stim_intact_moving.query('angle == @k and timepoints > 3 and filename == @file').timepoints,stim_intact_moving.query('angle == @k and timepoints > 3 and filename == @file').value,label=ang)#,c=cmapz[3-i])
        
        difz = stim_intact_moving.query('angle == @k and timepoints > 3 and filename == @file').value.diff()
        ax2.plot(stim_intact_moving.query('angle == @k and timepoints > 3 and filename == @file').timepoints,difz,label=ang)#,c=cmapz[3-i])

# Divide data into stance and swing phase

### Peakfinder in the angle traces 

In [ ]:
## need to use the change in direction of angle 1 (coxa-femur) to define the transition from swing to stance.
## use the change in direction of angle 2 (femur-tibia) to define transition from stance to swing. (These are from observations of the videos)

In [ ]:
def peaks_troughs(dataz,peak_prom = 10,trough_ord = 20,graph = 'yes',title = "",threshold = 0,remove = 'no'): #remove is to remove the first point in the line of the swing, which is messing up the averaging
    merged_all = pd.DataFrame(columns = ['timepoints', 'phase', 'cycle', 'filename', 'value', 'angle', 'filename_y'])
    for filez in dataz.filename.unique():
        data_1 = dataz.query('filename == @filez and angle == 1').value
        times_1 = np.array(dataz.query('filename == @filez and angle == 1').timepoints)
        peaks = scisig.find_peaks(data_1,height = 0, prominence = peak_prom)
        peaks_locs = peaks[0]
        peaks_heights = peaks[1]["peak_heights"]
        if threshold > 0:
            locations = peaks_heights > threshold
            peaks_locs = peaks_locs[locations]
            peaks_heights = peaks_heights[locations]
        scatterz = [times_1[x] for x in peaks_locs]
        peaks_times = scatterz
        
        data_2 = dataz.query('filename == @filez and angle == 2').value
        times_2 = np.array(dataz.query('filename == @filez and angle == 2').timepoints)
        troughs = scisig.argrelmin(data_2.values,order = trough_ord)
        scatterz2_height = [data_2.values[x] for x in troughs[0]]
        scatterz2 = [times_2[x] for x in troughs[0]]
        troughs_times = scatterz2
        
        ps_ts = pd.DataFrame(list(zip(peaks_heights,peaks_times)),columns = ["height","Times"])
        ps_ts["type"] = "peak"
        ps_tst = pd.DataFrame(list(zip(scatterz2_height,troughs_times)),columns = ["height","Times"])
        ps_tst["type"] = "trough"
        ps_ts = pd.concat([ps_ts,ps_tst])
        
        cleaned = smooth_peak_trough(ps_ts)
        
        if graph == 'yes':
            fig2,ax2 = plt.subplots(figsize=(10,4))
            plt.plot(times_1,data_1,label = "angle1",zorder = 1,color='k')
            plt.plot(times_2,data_2,label = "angle2",zorder = 1,color=[0.5,0.5,0.5])
            
            plt.scatter(cleaned.query("type == 'peak'").Times,cleaned.query("type == 'peak'").height,color = [1,0.4,0.4],label = "Peak")
            plt.scatter(cleaned.query("type == 'trough'").Times,cleaned.query("type == 'trough'").height,color = [0.2,0.7,0.7],label = "Trough")
            
            
            plt.title(filez,fontsize=10)
            plt.xlabel("Time in s")
            plt.ylabel("Angle 1")
            plt.legend()
        
        
        phase = pd.DataFrame(columns = ["timepoints","phase","cycle"])
        cycle=0
        for row in range(cleaned.shape[0]-1):
            #print(row)
            time1 = cleaned.iloc[row].Times
            time2 = cleaned.iloc[row+1].Times
            if cleaned.iloc[row].type == "trough":
                #print("swing")
                if remove == 'yes':
                    swingz = np.array(dataz.query("filename == @filez and timepoints > @time1 and timepoints <= @time2 and angle == 1").sort_values("timepoints").timepoints)
                    swingz = pd.DataFrame(swingz[1:],columns = ["timepoints"])
                else:
                    swingz = pd.DataFrame(dataz.query("filename == @filez and timepoints > @time1 and timepoints <= @time2 and angle == 1").sort_values("timepoints").timepoints)
                swingz["phase"] = "swing"
                swingz["cycle"] = cycle
                swingz["filename"] = filez
                cycle+=1

                phase = pd.concat([phase,swingz])
            else:
                #print("")
                #print("stance")
                #print(ang1.query("timepoints > @time1 and timepoints <= @time2").timepoints)
                stancez = pd.DataFrame(dataz.query("filename == @filez and timepoints > @time1 and timepoints <= @time2 and angle == 1").sort_values("timepoints").timepoints)
                stancez["phase"] = "stance"
                stancez["cycle"] = cycle
                stancez["filename"] = filez
                cycle+=1
                phase = pd.concat([phase,stancez])
        
        #print(phase.iloc[0:50])
        merged = phase.merge(dataz,on = "timepoints")
        merged = merged.drop_duplicates()
        merged = merged.rename(columns={"filename_x":"filename"})
        #print(list(merged))
        merged_all = pd.concat([merged_all,merged])
        
        
        
        
    return merged_all# #peaks_times,peaks_heights,troughs_times,scatterz2_height

In [ ]:
def smooth_peak_trough(data): # smooth the data so there can never be more than one peak or trough together
    sort_ps_ts = data.sort_values("Times")
    extraz = []
    allz = []
    for line in range(sort_ps_ts.shape[0]-1):
        typez = sort_ps_ts.iloc[line].type
        i = 1
        builder = []
        builder.append(line)
        while i > 0:
            #print(builder)
            try:
                if sort_ps_ts.iloc[line].type == sort_ps_ts.iloc[line+i].type:
                    builder.append(line+i)
                    i = i+1
                else: 
                    i = 0
                    if len(builder) > 1:
                        if len(allz) == 0:
                            allz.append(builder)
                        elif builder[0] not in allz[-1]:
                            allz.append(builder)
            except:
                i = 0
    
    sort_ps_ts.reset_index(inplace=True)
    #print(sort_ps_ts)
    #print(allz)
    
    to_remove = []
    for sequence in allz:
        compare = sort_ps_ts.iloc[sequence[0]:sequence[-1]+1]
        if compare.iloc[0].type == "trough":
            #print("extreme")
            extreme = compare.height.idxmin()#axis='columns')
            #print(extreme)
        else:
            extreme = compare.height.idxmax()
    
        sequence.remove(extreme) #select only the non-extreme values
        for seq in sequence:
            to_remove.append(seq)
    #print(len(to_remove))
    cleaned = sort_ps_ts.drop(to_remove)
    
    return cleaned

### Show the angle graphs with the peaks and the troughs

In [ ]:
merged_all = peaks_troughs(angles_walking,graph = 'yes',peak_prom=13,trough_ord = 9,remove = 'no')
merged_all_less = peaks_troughs(angles_walking,graph = 'yes',peak_prom=13,trough_ord = 9,remove = 'yes')

In [ ]:
#print(angles_walking.filename.unique().shape)
for file in merged_all_less.filename.unique():
    #print(file)
    fig, ax = plt.subplots(figsize = [15,5])
    for i,ang in enumerate(angles): 
        k = i+1
        times = np.linspace(0,(merged_all_less.query('filename == @file and angle == @k').timepoints.max()-merged_all_less.query('filename == @file and angle == @k').timepoints.min()),merged_all_less.query('filename == @file and angle == @k').timepoints.shape[0])
        #print(times)
        #ax.plot(angles_walking.query('filename == @file and angle == @k').timepoints,angles_walking.query('filename == @file and angle == @k').value,label=ang)
        ax.plot(times,merged_all_less.query('filename == @file and angle == @k').value,label=ang)
        #print(merged_all.query('filename == @file and angle == @k').value.iloc[:100])
        #print(times[0:100])
    ax.set_title(file[50:])
    ax.set_xlabel("Time (s)",fontsize = 15)
    ax.set_ylabel("Joint angle (degrees)",fontsize = 15)
    ax.legend()
    #ax.set_xlim([])

In [ ]:
#merged_all_stim = peaks_troughs(intact_ATR,graph = 'yes',peak_prom=13,trough_ord = 9) 

In [ ]:
#merged_all_decap = peaks_troughs(decapz,graph = 'yes',peak_prom=13,trough_ord = 9)

### Plot stance and swing separately

In [ ]:
cmapper = 'PiYG'
brg = cm.get_cmap(cmapper,2)
cmapz = brg(range(2))

fig, ax = plt.subplots(figsize = [8,6])
for each in merged_all.filename.unique():
    for cyc in merged_all_less.cycle.unique():
        for j, k in enumerate(["stance","swing"]):
            ax.plot(merged_all_less.query("filename == @each and phase == @k and angle == 1 and cycle == @cyc").value,merged_all_less.query("filename == @each and phase == @k and angle == 2 and cycle == @cyc").value,c = cmapz[j],linewidth = 0.5,alpha = 0.5)
            ax.scatter(merged_all.query("filename == @each and phase == @k and angle == 1 and cycle == @cyc").value,merged_all.query("filename == @each and phase == @k and angle == 2 and cycle == @cyc").value,color = cmapz[j],linewidth = 0.1,s=20,alpha = 0.2)
                #ax[0].scatter(merged.query("phase == @k and angle == 1 and cycle == @cyc").value,merged.query("phase == @k and angle == 2 and cycle == @cyc").value,s=1,alpha=0.5)
ax.set_title("Joint angles while walking")
ax.set_xlabel("Coxa-Femur angle",fontsize = 12)
ax.set_ylabel("Femur-Tibia angle",fontsize = 12)
ax.set_xlim([0,125])
ax.set_ylim([0,180])
ax.legend(["Stance","Swing"])
    

In [ ]:
cmapper = 'PiYG'
brg = cm.get_cmap(cmapper,2)
cmapz = brg(range(2))


for each in stim_intact_moving.query("timepoints < 1 and timepoints > 0").filename.unique():
    fig, ax = plt.subplots(figsize = [8,6])
    
    #print(stim_intact_moving.query("filename == @each and angle == 1 and timepoints < 1 and timepoints > 0"))
    #print(stim_intact_moving.query("filename == @each and angle == 1 and timepoints < 1 and timepoints > 0").value.shape)
    
    ax.plot(stim_intact_moving.query("filename == @each and angle == 1 and timepoints < 1 and timepoints > 0").value,stim_intact_moving.query("filename == @each and angle == 2 and timepoints < 1 and timepoints > 0").value,c = cmapz[j],linewidth = 0.5,alpha = 0.5)
    ax.scatter(stim_intact_moving.query("filename == @each and angle == 1 and timepoints < 1 and timepoints > 0").value,stim_intact_moving.query("filename == @each and angle == 2 and timepoints < 1 and timepoints > 0").value,color = cmapz[j],linewidth = 0.1,s=20,alpha = 0.5)
    #ax[0].scatter(merged.query("phase == @k and angle == 1 and cycle == @cyc").value,merged.query("phase == @k and angle == 2 and cycle == @cyc").value,s=1,alpha=0.5)
    
    ax.set_title(each[50:])
    ax.set_xlabel("Coxa-Femur angle",fontsize = 12)
    ax.set_ylabel("Femur-Tibia angle",fontsize = 12)
    ax.set_xlim([0,125])
    ax.set_ylim([0,180])
    ax.legend(["Stance","Swing"])
    

### Bin the data to find the mean traces of the stance and swing phases

In [ ]:
stances_1 = np.array(merged_all.query("phase == 'stance' and angle == 1").value)
stances_2 = np.array(merged_all.query("phase == 'stance' and angle == 2").value)
swings_1 = np.array(merged_all.query("phase == 'swing' and angle == 1").value)
swings_2 = np.array(merged_all.query("phase == 'swing' and angle == 2").value)
swings_1_less = np.array(merged_all_less.query("phase == 'swing' and angle == 1").value)
swings_2_less = np.array(merged_all_less.query("phase == 'swing' and angle == 2").value)
stancez = pd.DataFrame(stances_1,columns = ["Angle1"])
stancez["Angle2"] = stances_2
swingz_less = pd.DataFrame(swings_1_less,columns = ["Angle1"])
swingz_less["Angle2"] = swings_2_less

bin_size = 9
bins = np.arange(0,190,bin_size)
labels = np.arange(0+bin_size/2,190-bin_size/2,bin_size)

stancez['bin'] = pd.cut(stancez['Angle1'], bins = bins,labels = labels)
#print(stancez)
stancez_mean = pd.pivot_table(data = stancez, index = 'bin', values = 'Angle2')
stancez_mean.reset_index(inplace = True)
#print(stancez_mean)

swingz_less['bin'] = pd.cut(swingz_less['Angle1'], bins = bins,labels = labels)
#print(stancez)
swingz_mean_less = pd.pivot_table(data = swingz_less, index = 'bin', values = 'Angle2')
swingz_mean_less.reset_index(inplace = True)


together = pd.concat([swingz_mean_less.iloc[1:-3],stancez_mean.iloc[::-1],swingz_mean_less.iloc[1:2]]) #cutting out the edges of the swingz because there are very few data points so it gets very noisy
#together_all = pd.concat([swingz_mean_less,stancez_mean.iloc[::-1],swingz_mean_less.iloc[0:1]])
#together_some = pd.concat([swingz_mean_less.iloc[1:-2],stancez_mean.iloc[::-1],swingz_mean_less.iloc[1:2]])
#print(together)

cmapper = 'PiYG'
brg = cm.get_cmap(cmapper,2)
cmapz = brg(range(2))

fig, ax = plt.subplots(figsize = [8,8])
for each in merged_all.filename.unique():
    for cyc in merged_all_less.cycle.unique():
        for j, k in enumerate(["stance","swing"]):
            ax.plot(merged_all_less.query("filename == @each and phase == @k and angle == 1 and cycle == @cyc").value,merged_all_less.query("filename == @each and phase == @k and angle == 2 and cycle == @cyc").value,c = cmapz[j],linewidth = 0.5,alpha = 0.4)
            ax.scatter(merged_all_less.query("filename == @each and phase == @k and angle == 1 and cycle == @cyc").value,merged_all_less.query("filename == @each and phase == @k and angle == 2 and cycle == @cyc").value,color = cmapz[j],linewidth = 0.1,s=20,alpha = 0.2)
            #ax[0].scatter(merged.query("phase == @k and angle == 1 and cycle == @cyc").value,merged.query("phase == @k and angle == 2 and cycle == @cyc").value,s=1,alpha=0.5)
ax.set_title("Joint angles while walking")
ax.set_xlabel("Coxa-Femur angle",fontsize = 12)
ax.set_ylabel("Femur-Tibia angle",fontsize = 12)
ax.set_xlim([-20,160])
ax.set_ylim([0,180])
ax.legend(["Stance","Swing"])
plt.plot(np.array(together["bin"]),np.array(together["Angle2"]),c="k")
#fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/swing-stance.svg")

cmapper = 'PiYG'
brg = cm.get_cmap(cmapper,2)
cmapz = brg(range(2))

fig, ax = plt.subplots(figsize = [8,7])
for each in merged_all.filename.unique():
    for cyc in merged_all_less.cycle.unique():
        for j, k in enumerate(["stance","swing"]):
            ax.plot(merged_all_less.query("filename == @each and phase == @k and angle == 1 and cycle == @cyc").value,merged_all_less.query("filename == @each and phase == @k and angle == 2 and cycle == @cyc").value,c = cmapz[j],linewidth = 0.5,alpha = 0.4)
            ax.scatter(merged_all_less.query("filename == @each and phase == @k and angle == 1 and cycle == @cyc").value,merged_all_less.query("filename == @each and phase == @k and angle == 2 and cycle == @cyc").value,color = cmapz[j],linewidth = 0.1,s=20,alpha = 0.2)
            #ax[0].scatter(merged.query("phase == @k and angle == 1 and cycle == @cyc").value,merged.query("phase == @k and angle == 2 and cycle == @cyc").value,s=1,alpha=0.5)
ax.set_title("Joint angles while walking")
ax.set_xlabel("Coxa-Femur angle",fontsize = 12)
ax.set_ylabel("Femur-Tibia angle",fontsize = 12)
ax.set_xlim([0,125])
ax.set_ylim([0,180])
ax.legend(["Stance","Swing"])
#plt.plot(np.array(together["bin"]),np.array(together["Angle2"]),c="k")

#plt.plot([0,60],[45,105])
#plt.plot([60,120],[105,125])
#plt.plot([0,120],[45,125])


#plt.plot(np.array(together_all["bin"]),np.array(together_all["Angle2"]),c="b")
#plt.plot(np.array(together_some["bin"]),np.array(together_some["Angle2"]),c="r")
#plt.scatter(stancez_mean["bin"],stancez_mean["Angle2"],c="k")
#plt.scatter(swingz_mean_less["bin"][2:-3],swingz_mean_less["Angle2"][2:-3],c="k")

fig, ax = plt.subplots(figsize = [4,3])
for e in range(together.shape[0]-1):
    a = np.array([together.iloc[e].bin,together.iloc[e].Angle2])
    b = np.array([together.iloc[e+1].bin,together.iloc[e+1].Angle2])
    plt.plot([a[0],b[0]],[a[1],b[1]],linewidth = 4)



In [ ]:
cmapper = 'gist_rainbow'

brg = cm.get_cmap(cmapper,angles_walking.filename.unique().shape[0])
cmapz = brg(range(angles_walking.filename.unique().shape[0]))
fig, ax = plt.subplots(1,4,figsize = [22,5])

for each, i in enumerate(angles_walking.filename.unique()): #nb only use the first1s here for the line plot to be in line with the 1s used in the stimulated cases  
    joint_angle_line(angles_walking.query('filename == @i'),angles_walking.query('filename == @i').timepoints.max(),angles_walking.query('filename == @i').timepoints.min(),col = cmapz[each],ax_n1 = 0,alph=1/(angles_walking.shape[0])*700)
    
ax[0].plot(np.array(together["bin"]),np.array(together["Angle2"]),c="k")        
ax[0].set_title("Joint angles while walking ")
ax[0].set_xlabel("Coxa-Femur angle",fontsize = 12)
ax[0].set_ylabel("Femur-Tibia angle",fontsize = 12)
ax[0].set_xlim([-20,160])
ax[0].set_ylim([0,180])

brg = cm.get_cmap(cmapper,stim_intact_moving.filename.unique().shape[0]+1)
cmapz = brg(range(stim_intact_moving.filename.unique().shape[0]+1))

for each, i in enumerate(stim_intact_moving.filename.unique()):
    joint_angle_line(stim_intact_moving.query('filename == @i'),1,0,col = cmapz[each],ax_n1 = 1)

ax[1].plot(np.array(together["bin"]),np.array(together["Angle2"]),c="k")       
#ax[1,0].set_title("Joint angles after stimulation (first 1 s)")
ax[1].set_xlabel("Coxa-Femur angle",fontsize = 12)
ax[1].set_ylabel("Femur-Tibia angle",fontsize = 12)
ax[1].set_xlim([-20,160])#
ax[1].set_ylim([0,180])


brg = cm.get_cmap(cmapper,stim_intact_moving.query('timepoints > 3').filename.unique().shape[0]+1)
#print(stim_intact_moving.filename.unique().shape[0])
cmapz = brg(range(stim_intact_moving.query('timepoints > 3').filename.unique().shape[0]+1))
for each, i in enumerate(stim_intact_moving.query('timepoints > 3').filename.unique()):
    joint_angle_line(stim_intact_moving.query('filename == @i'),5,3,col = cmapz[each],ax_n1 = 2)


ax[2].plot(np.array(together["bin"]),np.array(together["Angle2"]),c="k")       
#ax[1,1].set_title("Joint angles after stimulation (3-5s)")
ax[2].set_xlabel("Coxa-Femur angle",fontsize = 12)
ax[2].set_ylabel("Femur-Tibia angle",fontsize = 12)
ax[2].set_xlim([-20,160])
ax[2].set_ylim([0,180])

brg = cm.get_cmap(cmapper,stim_decap_moving.filename.unique().shape[0]+1)
cmapz = brg(range(stim_decap_moving.filename.unique().shape[0]+1))

for each, i in enumerate(stim_decap_moving.filename.unique()):
    joint_angle_line(stim_decap_moving.query('filename == @i'),1,0,col = cmapz[each],ax_n1 = 3)
ax[3].plot(np.array(together["bin"]),np.array(together["Angle2"]),c="k")       
#ax[2,0].set_title("Joint angles after stimulation decapitated (first 1 s)")
ax[3].set_xlabel("Coxa-Femur angle",fontsize = 12)
ax[3].set_ylabel("Femur-Tibia angle",fontsize = 12)
ax[3].set_xlim([-20,160])
ax[3].set_ylim([0,180])

#hist = ax[1].hist2d(stim_decap_moving.query('angle == 1 and timepoints > 0 and timepoints < @poststim').value,stim_decap_moving.query('angle == 2 and timepoints > 0 and timepoints < @poststim').value,bins=[bins1,bins2],density=True)#vmax=18)
#ax[1].set_xlim([0,125])
#ax[1].set_ylim([0,180])


#for each, i in enumerate(stim_decap_moving.filename.unique()):
#    joint_angle_line(stim_decap_moving.query('filename == @i'),5,3,col = cmapz[each],ax_n1 = 2,ax_n2 = 1)
#ax[2,1].plot(np.array(together["bin"]),np.array(together["Angle2"]),c="k")       
#ax[2,1].set_title("Joint angles after stimulation decapitated (3-5s)")
#ax[2,1].set_xlabel("Coxa-Femur angle",fontsize = 12)
#ax[2,1].set_ylabel("Femur-Tibia angle",fontsize = 12)
#ax[2,1].set_xlim([0,125])
#ax[2,1].set_ylim([0,180])
#
##hist = ax[1].hist2d(stim_decap_moving.query('angle == 1 and timepoints > 3 and timepoints < 5').value,stim_decap_moving.query('angle == 2 and timepoints > 3 and timepoints < 5').value,bins=[bins1,bins2],density=True)#vmax=18)
##ax[1].set_xlim([0,125])
##ax[1].set_ylim([0,180])

fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/angle-angleplots-mean.svg")

### Interpolate points to make a near continuous line

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
to_add = pd.DataFrame({"bin":[12.5],"Angle2":[float(stancez_mean.iloc[:1].Angle2)]}) #adding an end point to join to
swingz_added = pd.concat([to_add,swingz_mean_less.iloc[1:-3],stancez_mean.iloc[-1:]])
interval = 0.2

f = interp1d(swingz_added.bin,swingz_added.Angle2)
desired_bins_swingz = np.arange(np.min(swingz_added.bin),np.max(swingz_added.bin)+interval,interval)
interpolated_swingz = f(desired_bins_swingz)

f = interp1d(stancez_mean.bin,stancez_mean.Angle2)
desired_bins_stancez = np.arange(np.min(stancez_mean.bin),np.max(stancez_mean.bin)+interval,interval)
interpolated_stancez = f(desired_bins_stancez)

both_int = pd.DataFrame({"Angle1":desired_bins_swingz,"Angle2":interpolated_swingz})
to_add = pd.DataFrame({"Angle1":desired_bins_stancez,"Angle2":interpolated_stancez})
both_int = pd.concat([both_int,to_add])

#print(interpolated_swingz.shape)
#print(interpolated_stancez.shape)

fig,ax1 = plt.subplots(figsize=(5,4))

plt.scatter(desired_bins_swingz,interpolated_swingz)
plt.scatter(desired_bins_stancez,interpolated_stancez)
plt.scatter(np.array(swingz_mean_less.iloc[1:-3].bin),swingz_mean_less.iloc[1:-3].Angle2)
plt.scatter(np.array(stancez_mean.iloc[::-1].bin),stancez_mean.iloc[::-1].Angle2)


In [ ]:
def distance_from_line(data,comparison,graph="no"):
    dataz = data.query("angle == 1 or angle == 2")
    #print(dataz)
    dataz = dataz.loc[:,['value','angle','timepoints','filename']]
    together = np.array((1,1))
    for file in dataz.filename.unique():
        #print(dataz.query("filename == @file"))
        dat_resh = dataz.query("filename == @file").pivot_table(index = "timepoints",columns = "angle")
        #print(dat_resh)
        if together.shape[0] == 2:
            together = dat_resh
        else:
            together = pd.concat([together,dat_resh])
    print(together.shape)
    smallest = []
    point_small = []
    for row in tqdm(together.iterrows()):
        #print(row[1])
        #print(row[1][0])
        distances = []
        points = []
        point_a = np.array((float(row[1][0]),float(row[1][1])))
        #print(point_a)
        #print(smallest)
        #print("")
        for rw in comparison.iterrows():
            point_b = np.array((float(rw[1].Angle1),float(rw[1].Angle2)))
            dif = np.linalg.norm(point_a-point_b)
            distances.append(dif)
            points.append(point_b)
            #print(points[0])
        smallest.append(np.min(distances))
        #print(np.where(distances == np.amin(distances))[0][0])
        point_small.append(points[np.where(distances == np.amin(distances))[0][0]])
    if graph == 'yes':
        cmapper = 'hsv'
        brg = cm.get_cmap(cmapper,len(smallest))
        cmapz = brg(range(len(smallest)))
        fig, ax = plt.subplots(figsize = [8,7])
        plt.scatter(comparison.Angle1,comparison.Angle2,s=1)
        for j, point in tqdm(enumerate(point_small)):
            #print(together.iloc[j])
            #print(together.iloc[j][0])
            #print(together.iloc[j][1])
            ax.scatter(together.iloc[j][0],together.iloc[j][1],color = cmapz[j],s=2)
            #print(point_small[j][0])
            #ax.scatter(point_small[j][0],point_small[j][1],color = cmapz[j])
        ax.set_xlim([0,180])
        ax.set_ylim([0,180])
    return(smallest,point_small)

In [ ]:
#print(angles_walking)

In [ ]:
distances_walking,locs_walking = distance_from_line(angles_walking,both_int,graph="yes")

In [ ]:
distances_stim_initial,locs_stim_initial = distance_from_line(stim_intact_moving.iloc[:,:-3].query("timepoints > 0 and timepoints < 1"),both_int,graph = "no")

In [ ]:
distances_stim_late,locs_stim_late = distance_from_line(stim_intact_moving.iloc[:,:-3].query("timepoints > 3 and timepoints < 5"),both_int,graph = "no")

In [ ]:
distances_decap,locs_decap = distance_from_line(stim_decap_moving.iloc[:,:-2].query("timepoints > 0 and timepoints < 1"),both_int,graph = "no")

### Histograms of distances from lines

In [ ]:
bins = np.arange(0,50,1.5)

to_shift = 0.2

fig1,ax1 = plt.subplots(figsize=(10,8))

hist_distances = np.histogram(distances_walking,bins = bins,density=True)#,range=[0,8])
ax1.bar(hist_distances[1][:-1]-to_shift,hist_distances[0],align='edge',linewidth = 2,label="Walking",color=walk,alpha=0.7)#,width=bin_size-bin_size/10,alpha = 0.9)
plt.plot(hist_distances[1][:-1],hist_distances[0],linewidth=5,color=walk)


hist_distances = np.histogram(distances_stim_late,bins = bins,density=True)#,bins = bins,range=[0,8])
ax1.bar(hist_distances[1][:-1]+to_shift,hist_distances[0],align='edge',linewidth = 2,label="Stim Late",color=sl,alpha=0.6)#,width=bin_size-bin_size/10,alpha = 0.9)
plt.plot(hist_distances[1][:-1]+to_shift*2,hist_distances[0],linewidth=5,color=sl)

hist_distances = np.histogram(distances_decap,bins = bins,density=True)#,bins = bins,range=[0,8])
ax1.bar(hist_distances[1][:-1]+to_shift*2,hist_distances[0],align='edge',linewidth = 2,label="Decap",color=dec,alpha=0.6)#,width=bin_size-bin_size/10,alpha = 0.9)
plt.plot(hist_distances[1][:-1]+to_shift*3,hist_distances[0],linewidth=5,color=dec)

hist_distances = np.histogram(distances_stim_initial,bins = bins,density=True)#,bins = bins,range=[0,8])
ax1.bar(hist_distances[1][:-1],hist_distances[0],align='edge',linewidth = 2,label="Stim Initial",color=si,alpha=0.5)#,width=bin_size-bin_size/10,alpha = 0.9)
plt.plot(hist_distances[1][:-1]+to_shift,hist_distances[0],linewidth=5,color=si)

ax1.set_title("Density",fontsize = 20)
ax1.set_xlabel("Distance from mean",fontsize=15)

ax1.legend()

fig1.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/histogram.svg")

In [ ]:
dists_all = pd.DataFrame(distances_walking,columns = ["Distance"])
dists_all["State"] = "Walking"
#print(dists_all)

dists_add = pd.DataFrame(distances_stim_initial,columns = ["Distance"])
dists_add["State"] = "Stim Initial"
dists_all = pd.concat([dists_all,dists_add])

dists_add = pd.DataFrame(distances_stim_late,columns = ["Distance"])
dists_add["State"] = "Stim Late"
dists_all = pd.concat([dists_all,dists_add])

dists_add = pd.DataFrame(distances_decap,columns = ["Distance"])
dists_add["State"] = "Decapitated"
dists_all = pd.concat([dists_all,dists_add])



In [ ]:
gyr = [walk,si,sl,dec]
sns.set_palette(gyr)
#my_colors = ListedColormap(sns.color_palette(gyr))

In [ ]:
fig1,ax1 = plt.subplots(figsize=(9,6))
sns.boxplot(data=dists_all,x="State",y="Distance")

fig1,ax1 = plt.subplots(figsize=(9,6))
sns.set_style("whitegrid")
sns.violinplot(data=dists_all,x="State",y="Distance",cut=0)
fig1.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/violin.svg")

post_dunn = spost.posthoc_dunn(dists_all,group_col="State",val_col = "Distance")
print("Post dunn =")
print(post_dunn)
print("")

## Rotate the data, divide the space into quadrants, rotate back

In [ ]:
def rotater(data,origin = (60,85),degrees = -45):
    p = []
    for file in data.filename.unique():
        dataz = data.query("filename == @file")
        for time in dataz.timepoints.unique():
            x = float(dataz.query("timepoints == @time and angle == 1").value)
            y = float(dataz.query("timepoints == @time and angle == 2").value)
            p.append((x,y))
    
    angle = np.deg2rad(degrees)
    R = np.array([[np.cos(angle), -np.sin(angle)],
                  [np.sin(angle),  np.cos(angle)]])
    o = np.atleast_2d(origin)
    p = np.atleast_2d(p)
    return np.squeeze((R @ (p.T-o.T) + o.T).T)


In [ ]:
rotated_walking = rotater(angles_walking)
rotated_stim_initial = rotater(stim_intact_moving.query("timepoints > 0 and timepoints < 1"))
rotated_stim_later = rotater(stim_intact_moving.query("timepoints > 3 and timepoints < 5"))
rotated_decap = rotater(stim_decap_moving.query("timepoints > 0 and timepoints < 1"))

In [ ]:
fig, ax = plt.subplots(figsize = [10,7])
for each in rotated_walking[0:]:
    #print(each)
    #print(each[0])
    plt.scatter(each[0],each[1],color=walk,alpha=0.7,s=1.8)
for each in rotated_stim_initial[0:]:
    #print(each)
    #print(each[0])
    plt.scatter(each[0],each[1],c=si,alpha=0.7,s=1.8)
for each in rotated_stim_later[0:]:
    #print(each)
    #print(each[0])
    plt.scatter(each[0],each[1],c=sl,alpha=0.7,s=1.8)
for each in rotated_decap[0:]:
    #print(each)
    #print(each[0])
    plt.scatter(each[0],each[1],c=dec,alpha=0.7,s=1.8)
plt.hlines(95,-20,180,colors=[0.5,0.5,0.5])
plt.vlines(60,0,180,colors=[0.5,0.5,0.5])


In [ ]:
def quadrant(data,x_bound = 60,y_bound = 95):
    u_l = 0 #upper left
    u_r = 0 #upper right
    l_l = 0 #lower left
    l_r = 0 #lower right 
    for each in data[0:]:
        #print(each)
        if each[0] <= x_bound and each[1] <= y_bound:
            l_l += 1
        elif each[0] <= x_bound and each[1] > y_bound:
            u_l += 1
        elif each[0] > x_bound and each[1] <= y_bound:
            l_r += 1
        elif each[0] > x_bound and each[1] > y_bound:
            u_r += 1
        #print(l_l,u_l,l_r,u_r)
    #print(l_l,u_l,l_r,u_r)
    tog = np.array([u_r,u_l,l_l,l_r])
    normed = tog/sum(tog)
    return(normed)

In [ ]:
quad_walking = quadrant(rotated_walking)
quad_stim_intact_initial = quadrant(rotated_stim_initial)
quad_stim_intact_later = quadrant(rotated_stim_later)
quad_stim_decap = quadrant(rotated_decap)

In [ ]:
print(quad_walking)

In [ ]:
#calculate the values for the rotated line
values = [60,0,60,180,-20,95,180,95]
angles = [1,2,1,2,1,2,1,2]
timepoints = [0,0,1,1,2,2,3,3]
filename = "one"

points = pd.DataFrame({"value":values,"angle":angles,"timepoints":timepoints,"filename":filename})
rotated_lines = rotater(points,degrees = 45)

print(points)
print(rotated_lines)

In [ ]:
#fig, ax = plt.subplots(figsize = [4,4])
#
#for file in stim_intact_moving.filename.unique():
#    for time in stim_intact_moving.query("filename == @file and timepoints > 0 and timepoints < 1").timepoints.unique():
#        plt.scatter(stim_intact_moving.query("filename == @file and timepoints == @time and angle == 1").value,stim_intact_moving.query("filename == @file and timepoints == @time and angle == 2").value,s=2,c=si)
#
#ax.set_xlim([-20,160])
#ax.set_ylim([0,180])
#
#plt.plot([rotated_lines[0][0],rotated_lines[1][0]],[rotated_lines[0][1],rotated_lines[1][1]],c=[0.8,0.8,0.8],linewidth=3)
#plt.plot([rotated_lines[2][0],rotated_lines[3][0]],[rotated_lines[2][1],rotated_lines[3][1]],c=[0.8,0.8,0.8],linewidth=3)        
#
##fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/scatter-quadrants-stimearly.svg")
#        
#fig, ax = plt.subplots(figsize = [4,4])
#for file in stim_intact_moving.filename.unique():
#    for time in stim_intact_moving.query("filename == @file and timepoints > 3 and timepoints < 5").timepoints.unique():
#        plt.scatter(stim_intact_moving.query("filename == @file and timepoints == @time and angle == 1").value,stim_intact_moving.query("filename == @file and timepoints == @time and angle == 2").value,s=2,c=sl)
#
#ax.set_xlim([-20,160])
#ax.set_ylim([0,180])
#
#plt.plot([rotated_lines[0][0],rotated_lines[1][0]],[rotated_lines[0][1],rotated_lines[1][1]],c=[0.8,0.8,0.8],linewidth=3)
#plt.plot([rotated_lines[2][0],rotated_lines[3][0]],[rotated_lines[2][1],rotated_lines[3][1]],c=[0.8,0.8,0.8],linewidth=3)
#
#fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/scatter-quadrants-stimlate.svg")
        
fig, ax = plt.subplots(figsize = [4,4])
for file in stim_decap_moving.filename.unique():
    for time in stim_decap_moving.query("filename == @file").timepoints.unique():
        plt.scatter(stim_decap_moving.query("filename == @file and timepoints == @time and angle == 1").value,stim_decap_moving.query("filename == @file and timepoints == @time and angle == 2").value,s=2,c=dec)

ax.set_xlim([-20,160])
ax.set_ylim([0,180])

plt.plot([rotated_lines[0][0],rotated_lines[1][0]],[rotated_lines[0][1],rotated_lines[1][1]],c=[0.8,0.8,0.8],linewidth=3)
plt.plot([rotated_lines[2][0],rotated_lines[3][0]],[rotated_lines[2][1],rotated_lines[3][1]],c=[0.8,0.8,0.8],linewidth=3)

fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/scatter-quadrants-decap.svg")
        
##fig, ax = plt.subplots(figsize = [4,4])
##for file in angles_walking.filename.unique():
##    for time in angles_walking.query("filename == @file").timepoints.unique():
##        plt.scatter(angles_walking.query("filename == @file and timepoints == @time and angle == 1").value,angles_walking.query("filename == @file and timepoints == @time and angle == 2").value,s=2,c=walk)
##
##ax.plot(np.array(together["bin"]),np.array(together["Angle2"]),c="k")        
##ax.set_xlim([-20,160])
##ax.set_ylim([0,180])
##
##plt.plot([rotated_lines[0][0],rotated_lines[1][0]],[rotated_lines[0][1],rotated_lines[1][1]],c=[0.8,0.8,0.8],linewidth=3)
##plt.plot([rotated_lines[2][0],rotated_lines[3][0]],[rotated_lines[2][1],rotated_lines[3][1]],c=[0.8,0.8,0.8],linewidth=3)
##   
#fig.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/scatter-quadrants-walk.svg")

In [ ]:
fig, ax = plt.subplots(figsize = [8,8])
plt.axis('off')
ax.plot(np.array(together["bin"]),np.array(together["Angle2"]),c="k")
ax.set_xlim([-20,160])
ax.set_ylim([0,180])


#ax = plt.subplot(111, projection='polar')
##plt.axis('off')
#r = 0.12
#ax.bar([pi/2-r*1.5,pi-r*1.5,pi+pi/2-r*1.5,2*pi-r*1.5],quad_walking,alpha=0.8,width=pi/3.5,color=walk)
#ax.bar([pi/2-r*0.5,pi-r*0.5,pi+pi/2-r*0.5,2*pi-r*0.5],quad_stim_intact_initial,alpha=0.7,width=pi/3.5,color=si)
#ax.bar([pi/2+r*0.5,pi+r*0.5,pi+pi/2+r*0.5,2*pi+r*0.5],quad_stim_intact_later,alpha=0.6,width=pi/3.5,color=sl)
#ax.bar([pi/2+r*1.5,pi+r*1.5,pi+pi/2+r*1.5,2*pi+r*1.5],quad_stim_decap,alpha=0.5,width=pi/3.5,color=dec)
##ax.set_rticks(["Upper left","Upper Right","Lower left","Lower right"])
#ax.set_xticklabels(['', '', '', '', '', '', '', ''])
#ax.xaxis.grid(visible=None,which="major")
##ax.set_yticklabels([])
##ax.set_yticks([0, 1000, 2000, 3000])
#
## Remove spines
#ax.spines["start"].set_color("none")
#ax.spines["polar"].set_color("none")
#

#ax.figure.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/mean_overlay.svg")

In [ ]:
plt.figure(figsize = (8, 8))
ax = plt.subplot(111, projection='polar')
#plt.axis('off')
r = 0.12
ax.bar([pi/2-r*1.5,pi-r*1.5,pi+pi/2-r*1.5,2*pi-r*1.5],quad_walking,alpha=0.8,width=pi/3.5,color=walk)
ax.bar([pi/2-r*0.5,pi-r*0.5,pi+pi/2-r*0.5,2*pi-r*0.5],quad_stim_intact_initial,alpha=0.7,width=pi/3.5,color=si)
ax.bar([pi/2+r*0.5,pi+r*0.5,pi+pi/2+r*0.5,2*pi+r*0.5],quad_stim_intact_later,alpha=0.6,width=pi/3.5,color=sl)
ax.bar([pi/2+r*1.5,pi+r*1.5,pi+pi/2+r*1.5,2*pi+r*1.5],quad_stim_decap,alpha=0.5,width=pi/3.5,color=dec)
#ax.set_rticks(["Upper left","Upper Right","Lower left","Lower right"])
ax.set_xticklabels(['', '', '', '', '', '', '', ''])
ax.xaxis.grid(visible=None,which="major")
#ax.set_yticklabels([])
#ax.set_yticks([0, 1000, 2000, 3000])

# Remove spines
ax.spines["start"].set_color("none")
ax.spines["polar"].set_color("none")

ax.figure.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/circular-hist.svg")

In [ ]:
fig, ax = plt.subplots(figsize = [4,4])
#for file in angles_walking.filename.unique():
#    for time in angles_walking.query("filename == @file").timepoints.unique():
#        plt.scatter(angles_walking.query("filename == @file and timepoints == @time and angle == 1").value,angles_walking.query("filename == @file and timepoints == @time and angle == 2").value,s=2,c=walk)

ax.plot(np.array(together["bin"]),np.array(together["Angle2"]),c="k")        
ax.set_xlim([-20,160])
ax.set_ylim([0,180])

plt.plot([rotated_lines[0][0],rotated_lines[1][0]],[rotated_lines[0][1],rotated_lines[1][1]],c=[0.8,0.8,0.8],linewidth=3)
plt.plot([rotated_lines[2][0],rotated_lines[3][0]],[rotated_lines[2][1],rotated_lines[3][1]],c=[0.8,0.8,0.8],linewidth=3)

ax.figure.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/lines+mean.svg")

In [ ]:
quadrants = pd.DataFrame([quad_walking,quad_stim_intact_initial,quad_stim_intact_later,quad_stim_decap],columns = ["Upper right","Upper left","Lower left","Upper right"])

In [ ]:
quad_norm = quadrants/quadrants.loc[0]
print(quad_norm)

In [ ]:
plt.figure(figsize = (8, 8))
ax = plt.subplot(111, projection='polar')
#plt.axis('off')
r = 0.12
ax.bar([pi/2-r*1.5,pi-r*1.5,pi+pi/2-r*1.5,2*pi-r*1.5],quad_norm.loc[0],alpha=0.8,width=pi/3.5,color=walk)
ax.bar([pi/2-r*0.5,pi-r*0.5,pi+pi/2-r*0.5,2*pi-r*0.5],quad_norm.loc[1],alpha=0.7,width=pi/3.5,color=si)
ax.bar([pi/2+r*0.5,pi+r*0.5,pi+pi/2+r*0.5,2*pi+r*0.5],quad_norm.loc[2],alpha=0.6,width=pi/3.5,color=sl)
ax.bar([pi/2+r*1.5,pi+r*1.5,pi+pi/2+r*1.5,2*pi+r*1.5],quad_norm.loc[3],alpha=0.5,width=pi/3.5,color=dec)
#ax.set_rticks(["Upper left","Upper Right","Lower left","Lower right"])
ax.set_xticklabels(['', '', '', '', '', '', '', ''])
ax.xaxis.grid(visible=None,which="major")
#ax.set_yticklabels([])
#ax.set_yticks([0, 1000, 2000, 3000])

# Remove spines
ax.spines["start"].set_color("none")
ax.spines["polar"].set_color("none")


ax.figure.savefig("C:/Users/Anna/Documents/Science/MendesLab/codes/Xana/tracking_csvs/circular-hist.svg")